<a href="https://colab.research.google.com/github/respwill/disaster_tweet/blob/main/Tweet_disaster_classifier3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchmetrics -q
!pip install transformers -q
!pip install ray[tune] -q
!pip install wandb -q
!pip install lightning -q
!pip install contractions -q
!pip install unidecode -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.6/303.6 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, random_split, Dataset, Subset
# from torchtext import data
# from torchvision import transforms, models
from torchmetrics import Accuracy, F1Score
import lightning as pl
# from lightning.utilities.model_summary import ModelSummary
from lightning.pytorch.callbacks import ModelSummary
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import WandbLogger
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingLR
import wandb
import torchmetrics
import sys
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray import tune
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
import numpy as np
from transformers import T5ForConditionalGeneration, T5Tokenizer, get_linear_schedule_with_warmup



In [ ]:
wandb.init(
    #   mode='disabled',
      # Set the project where this run will be logged
      project="disaster_tweet_classification2",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"Test55",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.0001,
      "data_size": 1,
      "batch_size":32,
      "epochs":10,
      "weight_decay":1e-3,
        # "model_name":"distilbert-base-uncased" #"cardiffnlp/twitter-xlm-roberta-base-sentiment" #
       "model_name":"t5-base"
        # "model_name":"distilbert/distilbert-base-uncased-finetuned-sst-2-english"
      })
wandb_logger = WandbLogger()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

In [ ]:
import contractions
from bs4 import BeautifulSoup
from unidecode import unidecode
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

class textDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length, train_flag=True):
        self.stop_words = set(stopwords.words('english'))
        self.lemmatizer = WordNetLemmatizer()

        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.tweet = pd.DataFrame()
        self.tweet['tweet'] = self.texts
        self.tweets = self.text_preprocessing(self.texts)
        self.tweets['target'] = self.labels

    def rem_urls(self, data):
        regex = re.compile(f"https?://(www\.)?(\w+)(\.\w+)(/\w*)?")
        data = re.sub(regex, "", data)
        return data

    def rem_emails(self, data):
        regex = re.compile("([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")
        data = re.sub(regex, "", data)
        return data

    def rem_mention(self, data):
        regex = re.compile('@\w+')
        data = re.sub(regex, '', data)
        return data

    def rem_accent(self, data):
        data = unidecode(data)
        return data

    def rem_unicode(self, data):
        data = data.encode("ascii", "ignore").decode()
        return data

    def rem_punc(self, data):
        data = re.sub(f"[{string.punctuation}]", " ", data)
        return data

    def clean_numbers(self, data):
        data = re.sub('[0-9]{5,}', '#'*5, data)
        data = re.sub('[0-9]{4}', '#'*4, data)
        data = re.sub('[0-9]{3}', '#'*3, data)
        data = re.sub('[0-9]{2}', '#'*2, data)
        return data


    def rem_stopwords(self, data):
        return " ".join([word for word in str(data).split() if word not in self.stop_words])

    def rem_extra_space(self, data):
        data = re.sub(' +', ' ', data).strip()
        return data


    def lemmatize_data(self, data):
        words = [self.lemmatizer.lemmatize(word) for word in data.split()]
        data = ' '.join(words)
        return data

    def text_preprocessing(self, data):
        data.loc[:,'tweet'] = data['tweet'].str.lower()
        data.loc[:,'tweet'] = data['tweet'].apply(contractions.fix)
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_urls)
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_emails)
        data.loc[:,'tweet'] = data['tweet'].apply(lambda x: BeautifulSoup(x).get_text())
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_mention)
        data.loc[:,'tweet'] = data['tweet'].str.replace(':\(', 'sadness ')
        data.loc[:,'tweet'] = data['tweet'].str.replace(r':\)[$|\s]*', 'happiness ')
        data.loc[:,'tweet'] = data['tweet'].str.replace(r'\;\)[$|\s]*', 'happiness ')
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_accent)
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_unicode)
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_punc)
        data.loc[:,'tweet'] = data['tweet'].apply(self.clean_numbers)
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_stopwords)
        data.loc[:,'tweet'] = data['tweet'].apply(self.rem_extra_space)
        data.loc[:,'tweet'] = data['tweet'].apply(self.lemmatize_data)

        return data

    def __len__(self):
        return len(self.tweets)

    def __getitem__(self, idx):
        # input
        text_idx = self.tweets.columns.tolist().index('tweet')
        self.tweets.iloc[idx, [text_idx]] = self.tweets.iloc[idx, [text_idx]].apply(lambda x: 'Classify it either disaster or non-disaster: ' + x)
        text = self.tweets.iloc[idx, text_idx]
        encoding = self.tokenizer([text], return_tensors="pt", max_length=128, padding="max_length", truncation=True)
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        # output
        label_idx = self.tweets.columns.tolist().index('target')
        label = self.tweets.iloc[idx, label_idx]
        # targets = self.tokenizer(label, return_tensors="pt", max_length=128, pad_to_max_length=True)
        targets = self.tokenizer([label], return_tensors="pt", max_length=64, padding="max_length", truncation=True)
        labels = targets['input_ids'].flatten()
        label_attention_mask = targets['attention_mask'].flatten()

        return {'input_ids':input_ids, 'attention_mask':attention_mask, 'labels_ids':labels, 'label_attention_mask':label_attention_mask,  'true_label':label}

In [ ]:
class textClassifier(pl.LightningModule):
    def __init__(self, vocab_size=10_000, learning_rate=3e-4, cosine_t_max=100):
        super().__init__()
        self.save_hyperparameters()
        print(self.hparams)
        self.train_accuracy = Accuracy(task='binary')
        self.val_accuracy = Accuracy(task='binary')
        self.test_accuracy = Accuracy(task='binary')
        self.train_f1 = F1Score(task='binary', num_classes=1)
        self.val_f1 = F1Score(task='binary', num_classes=1)
        self.test_f1 = F1Score(task='binary', num_classes=1)
        self.config = AutoConfig.from_pretrained(wandb.config['model_name'])
        self.config.num_labels = 1
        self.config.cosine_t_max = cosine_t_max
        self.model = T5ForConditionalGeneration.from_pretrained(wandb.config['model_name'])
        self.tokenizer = T5Tokenizer.from_pretrained(wandb.config['model_name'])

    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None):
        return self.model(input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels)

    def _shared_step(self, batch):
        lm_labels = batch["labels_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100 #it improves the result of T5 model

        outputs = self(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            labels=lm_labels,
            # decoder_input_ids=batch["labels_ids"],
            # decoder_input_ids=lm_labels,
            # decoder_attention_mask=batch['label_attention_mask']
    )

        loss = outputs.loss

        # pred = self.model.generate(batch["input_ids"])
        # predicted_labels = self.tokenizer.batch_decode(pred, skip_special_tokens=True)

        # true_labels = batch["true_label"]

        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss = self._shared_step(batch)
        # decoded_pred = self.tokenizer.decode(predicted_labels, skip_special_tokens=True)
        # decoded_true = self.tokenizer.decode(true_labels, skip_special_tokens=True)
        # self.train_accuracy(decoded_pred, decoded_true)
        # self.train_f1(decoded_pred, decoded_true)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=len(batch))
        # self.log('train_acc', self.train_accuracy, on_step=True, on_epoch=True, prog_bar=True)
        # self.log('train_f1', self.train_f1, on_step=True, on_epoch=True, prog_bar=True)

        return loss

    def validation_step(self, batch, batch_idx):
        with torch.no_grad():
            loss = self._shared_step(batch)
            # self.val_accuracy(decoded_pred, decoded_true)
            # self.val_f1(decoded_pred, decoded_true
            # decoded_true = self.tokenizer.decode(true_labels, skip_special_tokens=True)
            # labels_flattened = labels.view(-1)
            # predictions_flattened = predictions.view(-1)
            # self.val_accuracy(decoded_pred, decoded_true)
            # self.val_f1(decoded_pred, decoded_true)
            self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, batch_size=len(batch))
            # self.log('val_acc', self.val_accuracy, on_step=True, on_epoch=True, prog_bar=True)
            # self.log('val_f1', self.val_f1, on_step=True, on_epoch=True, prog_bar=True)
            return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.hparams.learning_rate, weight_decay=wandb.config['weight_decay'])
        sch = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=self.config.cosine_t_max)
        return [optimizer], [sch]

In [ ]:
class testDataset(textDataset):
    def __init__(self, data_dir, train_flag=False):
        super().__init__(data_dir, train_flag)

    def __getitem__(self, idx):
        text_idx = self.tweets.columns.tolist().index('tweet')
        text = self.tweets.iloc[idx, text_idx]
        encoding = self.tokenizer.encode_plus(text, return_tensors="pt", max_length=192, pad_to_max_length=True)
        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()

        return {'input_ids':input_ids, 'attention_mask':attention_mask}

In [ ]:
def train_val_dataset(dataset, train_split=0.75, val_split=0.25):
    train_idx, val_idx = train_test_split(list(range(len(dataset))), train_size=train_split, test_size=val_split, random_state=42)
    print(len(train_idx), len(val_idx))
    datasets = {'train':Subset(dataset, train_idx), 'val':Subset(dataset, val_idx)}
    return datasets

In [ ]:
class textDatamodule(pl.LightningDataModule):
    def __init__(self, train_dataset, val_dataset, test_dataset, batch_size, tokenizer):
        super().__init__()
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        self.batch_size = batch_size
        self.tokenizer = tokenizer

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)

# Checkpoint call back

In [ ]:
run_name = wandb.run.name
checkpoint_callback = pl.pytorch.callbacks.ModelCheckpoint(
    dirpath='/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4',
    filename=f'{run_name}-{{epoch}}-{{train_loss:.4f}}-{{val_loss:.4f}}',
    monitor='val_loss',
    mode='min',
    save_top_k=2
)

In [ ]:
early_stop_callback = EarlyStopping(monitor="val_loss", mode="min", patience=10)

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
texts = pd.read_csv('/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4/nlp-getting-started/train.csv')
texts['keyword'] = texts['keyword'].fillna('')
texts['tweet'] = texts['keyword'] + ' ' + texts['text']
texts['target'] = texts['target'].apply(lambda x: 'disaster' if (x==1) else 'non-disaster')
labels = texts[['target']]
texts = texts[['tweet']]

In [ ]:
def run_test(n_splits, texts, labels, batch_size, epochs, lr, max_length):
    # split train and test first
    tokenizer = AutoTokenizer.from_pretrained(wandb.config['model_name'])
    train_idx, val_idx = train_test_split(list(range(len(texts))), train_size=0.8, test_size=0.2, random_state=42)

    # test_texts, test_labels = texts.iloc[val_idx], labels.iloc[val_idx]
    # test_dataset = textDataset(test_texts, test_labels, tokenizer, max_length)

    # texts, labels = texts.iloc[train_idx], labels.iloc[train_idx]

    train_texts = texts.iloc[train_idx]
    val_texts = texts.iloc[val_idx]

    train_labels = labels.iloc[train_idx]
    val_labels = labels.iloc[val_idx]

    train_dataset = textDataset(train_texts, train_labels, tokenizer, max_length)
    val_dataset = textDataset(val_texts, val_labels, tokenizer, max_length)

    data_module = textDatamodule(train_dataset, val_dataset, val_dataset, wandb.config['batch_size'], tokenizer)
    # data_module = textDatamodule(batch_size=wandb.config['batch_size'])

    model = textClassifier(vocab_size=data_module.tokenizer.vocab_size,
                            learning_rate=wandb.config['learning_rate'],
                            cosine_t_max=wandb.config['epochs']
                            )

    # checkpoint = torch.load("/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4/Test46-epoch=26-train_loss=0.0118-val_loss=0.0228.ckpt",
    #                        map_location=torch.device('cuda'))
    # model.load_state_dict(checkpoint['state_dict'], strict=False)

    # model.to(torch.device('cuda'))

    checkpoint_callback = pl.pytorch.callbacks.ModelCheckpoint(
        dirpath='/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4',
        filename=f'{run_name}-{{epoch}}-{{train_loss:.4f}}-{{val_loss:.4f}}',
        monitor='val_loss',
        mode='min',
        save_top_k=2)

    trainer = pl.Trainer(accelerator='gpu',
                devices=1,
                max_epochs=wandb.config['epochs'],
                logger=wandb_logger,
                callbacks=[checkpoint_callback, early_stop_callback]
                )

    trainer.fit(model, data_module)

In [ ]:
max_length = 128
batch_size = wandb.config['batch_size']
epochs = wandb.config['epochs']
lr = wandb.config['learning_rate']

run_test(
    n_splits=3,
    texts=texts,
    labels=labels,
    batch_size=batch_size,
    epochs=epochs,
    lr=lr,
    max_length=max_length
)

# K fold cross validation

In [ ]:
def cross_validation_kfold(n_splits, texts, labels, batch_size, epochs, lr, max_length):
    # split train and test first
    tokenizer = AutoTokenizer.from_pretrained(wandb.config['model_name'])
    train_idx, val_idx = train_test_split(list(range(len(texts))), train_size=0.8, test_size=0.2, random_state=42)
    test_texts, test_labels = texts.iloc[val_idx], labels.iloc[val_idx]
    test_dataset = textDataset(test_texts, test_labels, tokenizer, max_length)

    texts, labels = texts.iloc[train_idx], labels.iloc[train_idx]
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    for fold, (train_idx, val_idx) in enumerate(kf.split(texts)):
        print(f'Fold {fold + 1}')
        # train_texts = [texts[i] for i in train_idx]
        # val_texts = [texts[i] for i in val_idx]
        # train_labels = [labels[i] for i in train_idx]
        # val_labels = [labels[i] for i in val_idx]

        train_texts = texts.iloc[train_idx]
        val_texts = texts.iloc[val_idx]
        train_labels = labels.iloc[train_idx]
        val_labels = labels.iloc[val_idx]

        train_dataset = textDataset(train_texts, train_labels, tokenizer, max_length)
        val_dataset = textDataset(val_texts, val_labels, tokenizer, max_length)

        data_module = textDatamodule(train_dataset, val_dataset, test_dataset, wandb.config['batch_size'], tokenizer)
        # data_module = textDatamodule(batch_size=wandb.config['batch_size'])

        model = textClassifier(vocab_size=data_module.tokenizer.vocab_size,
                              learning_rate=wandb.config['learning_rate'],
                              cosine_t_max=wandb.config['epochs']
                              )

        # checkpoint = torch.load("/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4/Test46-epoch=26-train_loss=0.0118-val_loss=0.0228.ckpt",
        #                        map_location=torch.device('cuda'))
        # model.load_state_dict(checkpoint['state_dict'], strict=False)

        # model.to(torch.device('cuda'))

        checkpoint_callback = pl.pytorch.callbacks.ModelCheckpoint(
            dirpath='/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4',
            filename=f'{run_name}-{{epoch}}-{{train_loss:.4f}}-{{val_loss:.4f}}',
            monitor='val_loss',
            mode='min',
            save_top_k=2)

        trainer = pl.Trainer(accelerator='gpu',
                    devices=1,
                    max_epochs=wandb.config['epochs'],
                    logger=wandb_logger,
                    callbacks=[checkpoint_callback, early_stop_callback]
                    )

        trainer.fit(model, data_module)

# Load data

In [ ]:
texts = pd.read_csv('/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4/nlp-getting-started/train.csv')
texts['keyword'] = texts['keyword'].fillna('')
texts['tweet'] = texts['keyword'] + ' ' + texts['text']
texts['target'] = texts['target'].apply(lambda x: 'disaster' if (x==1) else 'non-disaster')
labels = texts[['target']]
texts = texts[['tweet']]

# Train model

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(wandb.config['model_name'])
# tokenizer = T5Tokenizer.from_pretrained(wandb.config['model_name'], model_max_length=128)
max_length = 128
batch_size = wandb.config['batch_size']
epochs = wandb.config['epochs']
lr = wandb.config['learning_rate']

cross_validation_kfold(
    n_splits=3,
    texts=texts,
    labels=labels,
    batch_size=batch_size,
    epochs=epochs,
    lr=lr,
    max_length=max_length
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

<ipython-input-4-bcb487067ae5>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].str.lower()
<ipython-input-4-bcb487067ae5>:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tweet'] = data['tweet'].apply(contractions.fix)
<ipython-input-4-bcb487067ae5>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas

...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /content/drive/MyDrive/ColoradoBoulder/Machine Learning/Deep learning/week4 exists and is not empty.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name           | Type                       | Params | Mode 
----------------------------------------------------------------------
0 | train_accu

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2760: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]